In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%xmode plain

Exception reporting mode: Plain


# Import the data

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
print(train.shape)
print(test.shape)

(1460, 81)
(1459, 80)


Okay, so 1,460 rows of 80 independent variables.

<img src="http://i0.kym-cdn.com/photos/images/facebook/000/930/538/7cc.jpg" >

Let's take a look at the data dictionary. (I then looked at the data dictionary and wrote nothing about it because there are too many damn variables. According to [the guy that got this data together](https://ww2.amstat.org/publications/jse/v19n3/decock.pdf), there are 23 nominal, 23 ordinal, 14 discrete, and 20 continuous variables here.)

Let's merge the test and train datasets for EDA.

In [19]:
test['SalePrice'] = np.nan
df = pd.concat([train, test], axis=0)
train_ids = train['Id']
test_ids = test['Id']

In [20]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [21]:
df.shape

(2919, 81)

# Clean the data

Below, I define a function that checks our dataframe for missing values and prints them in descending order of missings per variable. I'll use this below while cleaning and imputing the data to check my progress.

In [113]:
def check_missings():
    return df.apply(pd.isnull).sum().sort_values(ascending=False).loc[ lambda series: series>0 ]

In [30]:
df['PoolQC'] = df['PoolQC'].replace({np.nan: 'No pool'})

In [57]:
df['MiscFeature'] = df['MiscFeature'].replace({np.nan: 'No misc. features'})
df['Alley'] = df['Alley'].replace({np.nan: 'No alley'})
df['Fence'] = df['Fence'].replace({np.nan: 'No fence'})
df['FireplaceQu'] = df['FireplaceQu'].replace({np.nan: 'No fireplace'})
df['GarageFinish'] = df['GarageFinish'].replace({np.nan: 'No garage'})
df['GarageCond'] = df['GarageCond'].replace({np.nan: 'No garage'})
df['GarageQual'] = df['GarageQual'].replace({np.nan: 'No garage'})
df['GarageType'] = df['GarageType'].replace({np.nan: 'No garage'})
df['BsmtCond'] = df['BsmtCond'].replace({np.nan: 'No basement'})
df['BsmtExposure'] = df['BsmtExposure'].replace({np.nan: 'No basement'})
df['BsmtQual'] = df['BsmtQual'].replace({np.nan: 'No basement'})
df['BsmtFinType2'] = df['BsmtFinType2'].replace({np.nan: 'No basement'})
df['BsmtFinType1'] = df['BsmtFinType1'].replace({np.nan: 'No basement'})
df['MasVnrType'] = df['MasVnrType'].replace({np.nan: 'No masonry'})

In [62]:
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)

In [61]:
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)

In [65]:
df['LotFrontage'].describe()

count    2433.000000
mean       69.305795
std        23.344905
min        21.000000
25%        59.000000
50%        68.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64

My guess is that those missings for `LotFrontage` are actually zeros, since the minimum is 21ft:

In [66]:
df['LotFrontage'] = df['LotFrontage'].fillna(0)

In [69]:
df['MSZoning'].value_count()

RL         2265
RM          460
FV          139
RH           26
C (all)      25
Name: MSZoning, dtype: int64

In [70]:
df['MSZoning'] = df['MSZoning'].fillna('RL')

In [72]:
df['Utilities'].describe()

count       2917
unique         2
top       AllPub
freq        2916
Name: Utilities, dtype: object

In [74]:
df['Utilities'] = df['Utilities'].fillna('AllPub')

In [76]:
df['BsmtHalfBath'].value_counts()

0.0    2742
1.0     171
2.0       4
Name: BsmtHalfBath, dtype: int64

In [77]:
df['BsmtHalfBath'] = df['BsmtHalfBath'].fillna(0)

In [79]:
df['BsmtFullBath'] = df['BsmtFullBath'].fillna(0)

In [80]:
df['Functional'].value_counts()

Typ     2717
Min2      70
Min1      65
Mod       35
Maj1      19
Maj2       9
Sev        2
Name: Functional, dtype: int64

In [81]:
df['Functional'] = df['Functional'].fillna('Typ')

In [82]:
df['GarageArea'].describe()

count    2918.000000
mean      472.874572
std       215.394815
min         0.000000
25%       320.000000
50%       480.000000
75%       576.000000
max      1488.000000
Name: GarageArea, dtype: float64

In [83]:
df['GarageArea'] = df['GarageArea'].fillna(df['GarageArea'].mean())

In [85]:
df['GarageCars'].value_counts()

2.0    1594
1.0     776
3.0     374
0.0     157
4.0      16
5.0       1
Name: GarageCars, dtype: int64

In [86]:
df['GarageCars'] = df['GarageCars'].fillna(2)

In [87]:
df['BsmtFinSF2'] = df['BsmtFinSF2'].fillna(df['BsmtFinSF2'].mean())

In [88]:
df['BsmtFinSF1'] = df['BsmtFinSF1'].fillna(df['BsmtFinSF1'].mean())

In [89]:
df['Electrical'].value_counts()

SBrkr    2671
FuseA     188
FuseF      50
FuseP       8
Mix         1
Name: Electrical, dtype: int64

In [90]:
df['Electrical'] = df['Electrical'].fillna('SBrkr')

In [91]:
df['Exterior2nd'].value_counts()

VinylSd    1014
MetalSd     447
HdBoard     406
Wd Sdng     391
Plywood     270
CmentBd     126
Wd Shng      81
Stucco       47
BrkFace      47
AsbShng      38
Brk Cmn      22
ImStucc      15
Stone         6
AsphShn       4
CBlock        3
Other         1
Name: Exterior2nd, dtype: int64

In [92]:
df['Exterior2nd'] = df['Exterior2nd'].fillna('VinylSd')

In [95]:
df['KitchenQual'].value_counts()

TA    1492
Gd    1151
Ex     205
Fa      70
Name: KitchenQual, dtype: int64

In [98]:
df['KitchenQual'] = df['KitchenQual'].fillna('TA')

In [106]:
df['Exterior1st'] = df['Exterior1st'].fillna(df['Exterior1st'].value_counts().index[0])

In [108]:
df['BsmtUnfSF'] = df['BsmtUnfSF'].fillna(df['BsmtUnfSF'].mean())

In [109]:
df['TotalBsmtSF'] = df['TotalBsmtSF'].fillna(df['TotalBsmtSF'].mean())

In [110]:
df['SaleType'] = df['SaleType'].fillna(df['SaleType'].value_counts().index[0])

We've taken care of all of our missings. True, some of the square-footage values may not make sense if the observations with missing values didn't have the object whose footage was being measured, but there are so few missings that it won't have any impact on our results.

# Explore the data

In [114]:
print('Data from years {} to {}.'.format(df['YrSold'].min(), df['YrSold'].max()))

Data from years 2006 to 2010.


Some questions:

* We'll need to incorporate `YrSold` for sure, since this period contains the Great Recession.
* Do prices fluctuate with the time of year?
* What interaction terms might we expect here?
* Does it matter if the garage was built after the house was built?
* What the hell is a 1.5 storey home??
* I bet `Neighborhood` will be a very important predictor.
* Would it be possible to augment this dataset with Census data, e.g., on race?
* I bet Iowa State University has a huge influence on housing prices.

# Some info about Ames, Iowa

* **Population**: 66,191 in 2016, in the top ten for the state of Iowa.
* **Density**: 2,436/$mi^2$
* **Major employers**: State/federal agencies
* Ranked in the top 15 *"Cities that have done the best since the recession"* by Bloomberg
* **Unemployment**: Ranked along with Boulder, CO as lowest unemployment rate in 2016.

Ames is a moderately-sized town smack-dab in the middle of Iowa, containing Iowa State University (ISU) and nestled between the state's major east-west and north-south highways. ISU employs almost 16,000 of the states 66,00 residents. The Iowa DoT is located in the city. Finally, Ames sees a lot of political activity by virtue of its being in Iowa. According to Wikipedia, the Ames Straw Poll was held in the city from 1979 through 2011 before being cancelled.

<iframe src="https://www.google.com/maps/embed?pb=!1m18!1m12!1m3!1d47420.89231618093!2d-93.6613958149767!3d42.02589893729839!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m3!1m2!1s0x87ee70624634a06b%3A0x273156083cc75200!2sAmes%2C+IA!5e0!3m2!1sen!2sus!4v1507261616184" width="600" height="450" frameborder="0" style="border:0" allowfullscreen></iframe>

Looking at the map, a number of things jump out at me:

* Iowa State University is just west of the center of town; I bet this has a big impact on adjacent and nearby properties.
* There are a number of parks and golf courses. The two largest appear to be Coldwater Golf Links and the Ada Hayden Heritage Park.
* The airport is located at the very southern end of town.
* The town is framed on the south and east by the highways US-30 (a state route) and I-35 (a federal route).
* I bet Ames gets a lot of through-traffic due to the highways. Are there touristy neighborhoods or anything of that kind?
* US-69 runs north-south through the town, just west of I-35.
* The town's major streets more or less follow a grid pattern, while within each grid square there are residential streets.
* The South Skunk River runs north-south on the east side of town, while the Squaw River runs northwest-southeast, meeting up with the South Skunk in the southeast corner of town
* Can we feature-engineer postal code?
* A railroad runs east-west through town. According to Wikipedia, 60-70 trains run through Ames per day, sometimes leading to traffic delays. This would presumably decrease home values nearby.

I found a map of the neighborhoods:

<img src="" >